<a href="https://www.kaggle.com/code/biocanter/public-titanic-xgb-top20-2?scriptVersionId=112824873" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import confusion_matrix

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
df_train = pd.read_csv("../input/titanic/train.csv")
df_test = pd.read_csv("../input/titanic/test.csv")
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
###analyze missing data
#train
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print (missing_data.head(20))
##test
total = df_test.isnull().sum().sort_values(ascending=False)
percent = (df_test.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print (missing_data.head(20))

             Total   Percent
Cabin          687  0.771044
Age            177  0.198653
Embarked         2  0.002245
PassengerId      0  0.000000
Survived         0  0.000000
Pclass           0  0.000000
Name             0  0.000000
Sex              0  0.000000
SibSp            0  0.000000
Parch            0  0.000000
Ticket           0  0.000000
Fare             0  0.000000
             Total   Percent
Cabin        327.0  0.367003
Age           86.0  0.096521
Fare           1.0  0.001122
PassengerId    0.0  0.000000
Pclass         0.0  0.000000
Name           0.0  0.000000
Sex            0.0  0.000000
SibSp          0.0  0.000000
Parch          0.0  0.000000
Ticket         0.0  0.000000
Embarked       0.0  0.000000
Survived       NaN       NaN


In [5]:
tiket_rep = pd.concat([df_train['Ticket'], df_test['Ticket']])
tiket_rep.value_counts()
a=tiket_rep.value_counts()
tiket=a.index
numero=[]
for h in range(0,216):
    numero.append(str(h))
dict_from_list = dict(zip(tiket[0:216], numero))
df_train['Ticket']=df_train['Ticket'].map(dict_from_list)
df_test['Ticket']=df_test['Ticket'].map(dict_from_list)
df_test['Ticket']=df_test['Ticket'].fillna(1000)
df_train['Ticket']=df_train['Ticket'].fillna(1000)
df_train['Ticket'] = df_train['Ticket'].astype(int)
df_test['Ticket'] = df_test['Ticket'].astype(int)

In [6]:
####missing data and drop column
df_train.drop(["Cabin","PassengerId",'Name'],inplace=True,axis=1)
df_test.drop(["Cabin","PassengerId",'Name'],inplace=True,axis=1)

In [7]:
#Replacing nulls by column mean
#age
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())
df_test['Age'] = df_test['Age'].fillna(df_test['Age'].mean())
###fare
df_train['Fare'] = df_train['Fare'].fillna(df_train['Fare'].mean())
df_test['Fare'] = df_test['Fare'].fillna(df_test['Fare'].mean())
####here with the most popular feature
df_train['Embarked']=df_train['Embarked'].fillna('S')
df_test['Embarked']=df_test['Embarked'].fillna('S')

In [8]:
#Mappping categorical to numeric
df_train['Sex']=df_train['Sex'].map({'male':0,'female':1})
df_test['Sex']=df_test['Sex'].map({'male':0,'female':1})
#Mappping categorical to numeric
df_train['Embarked']=df_train['Embarked'].map({'S':0,'C':1,'Q':2})
df_test['Embarked']=df_test['Embarked'].map({'S':0,'C':1,'Q':2})

In [9]:
##assign X and y
X=df_train.drop('Survived',axis=1)
y=df_train['Survived']

In [10]:
####I've started with a basic model  
###
###split data 80% train, 20% test
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)
##1st model
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

#Defining the parameters to search with GridSearchCV
param_grid = {
'n_estimators': range(20, 100,20),
'max_depth': range(3, 8),
'learning_rate': [.1,0.15,.2, .3, .4, .5],
'colsample_bytree': [.7, .8, .9, 1]
}

#Specifying our classifier
xgb = XGBClassifier()
#Searching for the best parameters
g_search = GridSearchCV(estimator = xgb, param_grid = param_grid,
cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)
#Fitting the model using best parameters found
g_search.fit(train_X, train_y)
#Printing the best parameters found
print(g_search.best_params_)
pred_xg = g_search.predict(val_X)
ac = accuracy_score(val_y,pred_xg)
print (ac)

{'colsample_bytree': 0.8, 'learning_rate': 0.15, 'max_depth': 3, 'n_estimators': 20}
0.8268156424581006


In [11]:
#1st model
#ac.- 0.86
xgb_1 = XGBClassifier(colsample_bytree=0.8, 
                    learning_rate= 0.15, 
                    max_depth= 3, 
                    n_estimators= 20)

xgb_1.fit(X, y)
pred_1 = xgb_1.predict(df_test)

In [12]:
##RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV

params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
              "min_child_weight" : [ 1, 3, 5, 7 ],
           'colsample_bytree': np.arange(0.2, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
              "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
           'n_estimators': [50,100, 500, 1000]}

#Specifying our classifier
xgb = XGBClassifier()
#Searching for the best parameters
g_search = RandomizedSearchCV(estimator = xgb, 
                              param_distributions = params, n_iter=25,
                            verbose=1)
#Fitting the model using best parameters found
g_search.fit(train_X, train_y)
#Printing the best parameters found
print(g_search.best_params_)
pred_xg = g_search.predict(val_X)
ac = accuracy_score(val_y,pred_xg)
print (ac)
##best model 0.877

Fitting 5 folds for each of 25 candidates, totalling 125 fits
{'subsample': 0.7, 'n_estimators': 50, 'min_child_weight': 1, 'max_depth': 20, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 0.9000000000000001, 'colsample_bylevel': 0.7}
0.8491620111731844


In [13]:
#2nd model.- 0.877
xgb_2 = XGBClassifier(subsample= 0.7,
                      n_estimators= 50,
                      min_child_weight= 1, 
                      max_depth= 5, 
                      learning_rate= 0.3, 
                      gamma=0.3, 
                      colsample_bytree=0.5, 
                      colsample_bylevel= 0.5)

xgb_2.fit(X, y)
pred_2 = xgb_2.predict(df_test)

In [14]:
###bayesian optimization
##install hyper 
!pip install hyperopt

In [15]:
###bayesian optimization 
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval 
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV

###bayesian optimization, more options
# Space, using choice
space = { 'subsample' : hp.choice('subsample', [0.2,0.4,0.6,1]),
    'max_depth': hp.choice("max_depth", range(3,21,3)),
        'gamma': hp.choice ('gamma', [i/10.0 for i in range(0,5)]),
         'reg_alpha' : hp.choice('reg_alpha', [1e-5, 1e-2, 0.1, 1, 10, 100]),               
        'reg_lambda' : hp.choice('reg_lambda', [1e-5, 1e-2, 0.1, 1, 10, 100]),                       
        'colsample_bytree' : hp.choice('colsample_bytree', [i/10.0 for i in range(3,10)]),
         'learning_rate' : hp.choice('learning_rate', [0.0001,0.001, 0.01, 0.1, 0.2,0.3,0.5,1]),
        'min_child_weight' : hp.choice('min_child_weight', [i/10.0 for i in range(0,10)]),
       'n_estimators': hp.choice("n_estimators", [5, 10, 20, 40, 80, 160,320,1000,3000,6000,8000])
    }
# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
# Objective function
def objective(params):
    
    xgboost = XGBClassifier(seed=0, **params)
    scores = cross_val_score(xgboost, train_X, train_y, cv=kfold, scoring='recall', n_jobs=-1)
    # Extract the best score
    best_score = max(scores)
    # Loss must be minimized
    loss = - best_score
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}
# Trials to track progress
bayes_trials = Trials()
# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 100, trials = bayes_trials)
# Print the index of the best parameters
print(best)
# Print the values of the best parameters
print(space_eval(space, best))

100%|██████████| 100/100 [10:28<00:00,  6.28s/trial, best loss: -0.7692307692307693]
{'colsample_bytree': 6, 'gamma': 2, 'learning_rate': 3, 'max_depth': 2, 'min_child_weight': 5, 'n_estimators': 7, 'reg_alpha': 1, 'reg_lambda': 5, 'subsample': 2}
{'colsample_bytree': 0.9, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 9, 'min_child_weight': 0.5, 'n_estimators': 1000, 'reg_alpha': 0.01, 'reg_lambda': 100, 'subsample': 0.6}


In [16]:
#bayesian optimization.- 1
xgb_3 = XGBClassifier(n_estimators= 160, 
                      min_child_weight= 0.8, 
                      max_depth= 9, 
                      learning_rate= 0.3, 
                      gamma=0.1, 
                      colsample_bytree=0.8, 
                      reg_alpha= 0.1,
                      reg_lambda= 1e-05,
                     subsample=0.6,
                     objective='binary:logistic')
xgb_3.fit(X, y)
pred_3 = xgb_3.predict(df_test)

In [17]:
#bayesian optimization.- 2
xgb_4 = XGBClassifier(subsample=0.4,
                           n_estimators= 320, 
                      min_child_weight= 0.2, 
                      max_depth= 3, 
                      learning_rate= 0.5, 
                      gamma=0.2, 
                      colsample_bytree=0.5, 
                      reg_alpha= 0.01,
                      reg_lambda= 0.1,
                     objective='binary:logistic')
xgb_4.fit(X, y)
pred_4 = xgb_4.predict(df_test)

In [18]:
##copy from https://www.kaggle.com/code/cbhavik/score-0-79665-titanic-xgb-forest-top-3
xgb_5 = XGBClassifier(learning_rate=0.005, ##learning rate
                       n_estimators=6000, ###
                       max_depth=4,
                       min_child_weight=0,
                       gamma=0.61,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       objective='reg:squarederror',
                       #nthread=-1,
                       #scale_pos_weight=1,
                       seed=27,
                       reg_alpha=0.00006,
                       random_state=42)

xgb_5.fit(X, y)
pred_5 = xgb_5.predict(df_test)

In [19]:
pred=np.stack((pred_1, pred_2,pred_3,pred_4,pred_5))

In [20]:
from scipy import stats as st
#print(st.mode(pred))
pred=st.mode(pred)
pred=pred[0]
pred = pred.flatten()
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [21]:
submission = pd.read_csv('../input/titanic/gender_submission.csv')
submission['Survived'] = np.round(pred).astype(int)
submission.to_csv('submission.csv',index=False)